# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-397c725c19-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Disney explorative search") 

Consider the following exploratory information need:

> investigate films produced by Disney and compare workers among films.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Walt Disney Animation Studios

3. Identify the BGP for Walt Disney (person)

4. List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

5. Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

6. Identify the BGP for the film "The Little Mermaid"

7. Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label). 

8. Identify the BGP for Academy Awards

9. For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [2]:
# query example
#i get instance of The Lion King
queryString = """
SELECT DISTINCT  ?o ?oname
WHERE {
   # bind something
    wd:Q36479 wdt:P31 ?o .
   # get the label
   ?o sc:name ?oname.
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q29168811'), ('oname', 'animated feature film')]
1


In [3]:
# query example
# i want to find film
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   wd:Q29168811 wdt:P279 ?s .
   ?s wdt:P279 ?o.
   # get the label
   #?s sc:name ?sname.
   ?o sc:name ?oname.
   #FILTER REGEX(?sname,'films','i')
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11424'), ('oname', 'film')]
1


Final query for this task

In [4]:
# query example
#i find the film Q11424
#i find bgp for film
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q11424 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
}

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pname', 'studied by'), ('o', 'http://www.wikidata.org/entity/Q1660187'), ('oname', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P345'), ('oname', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1040'), ('oname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P344'), ('oname', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pname', 'model item'), ('o', 'http://www.wikidata.org/entity/Q17738'), ('oname', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q55321'), ('oname', 'Category:Films

## Task 2
Identify the BGP for Walt Disney Animation Studios

In [5]:
# write your queries
# query example
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q36479 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   FILTER REGEX(?oname,'Walt Disney Animation Studios')
}

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pname', 'part of the series'), ('o', 'http://www.wikidata.org/entity/Q56070713'), ('oname', 'Walt Disney Animation Studios film')]
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('pname', 'production company'), ('o', 'http://www.wikidata.org/entity/Q1047410'), ('oname', 'Walt Disney Animation Studios')]
2


Final query for this task

In [14]:
# write your final query
# the bgp of Walt Disney Animation Studios 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q1047410 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   
}

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q8610'), ('oname', 'Academy Scientific and Technical Award')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q191224'), ('oname', 'Walt Disney Pictures')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by'), ('o', 'http://www.wikidata.org/entity/Q7414'), ('oname', 'The Walt Disney Company')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1762059'), ('oname', 'film production company')]
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('pname', 'director / manager'), ('o', 'http://www.wikidata.org/entity/Q5127380'), ('oname', 'Clark Spencer')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p',

In [7]:
# write your final query
 
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
   # bind something
   ?s wdt:P272 wd:Q1047410.
   # get the label
   ?s sc:name ?sname.
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q223163'), ('sname', "The Emperor's New Groove")]
[('s', 'http://www.wikidata.org/entity/Q18647981'), ('sname', 'Moana')]
[('s', 'http://www.wikidata.org/entity/Q179673'), ('sname', 'Beauty and the Beast')]
[('s', 'http://www.wikidata.org/entity/Q15046569'), ('sname', 'Roger Rabbit short films')]
[('s', 'http://www.wikidata.org/entity/Q3362144'), ('sname', 'Trail Mix-Up')]
[('s', 'http://www.wikidata.org/entity/Q379873'), ('sname', 'Who Framed Roger Rabbit')]
[('s', 'http://www.wikidata.org/entity/Q36479'), ('sname', 'The Lion King')]
[('s', 'http://www.wikidata.org/entity/Q215617'), ('sname', 'Sleeping Beauty')]
[('s', 'http://www.wikidata.org/entity/Q212792'), ('sname', 'Bolt')]
[('s', 'http://www.wikidata.org/entity/Q944287'), ('sname', 'The Little Mermaid')]
10


## Task 3
Identify the BGP for Walt Disney (person)

In [8]:
# write your queries
# I find the Walt Disney
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
   # bind something
   ?s wdt:P31 wd:Q5.
     # ?p ?o.
   # get the label
   ?s sc:name ?sname.
   FILTER REGEX(?sname,'Walt Disney')
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8704'), ('sname', 'Walt Disney')]
1


In [ ]:
# write your queries
#  I find the bgp for Walt Disney
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q8704 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Final query for this task

In [9]:
# write your final query
# we want to find the occupation of walt desiny
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   wd:Q8704 wdt:P106 ?o.
   # get the label
   ?o sc:name ?oname.
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3658608'), ('oname', 'caricaturist')]
[('o', 'http://www.wikidata.org/entity/Q1028181'), ('oname', 'painter')]
[('o', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'film actor')]
[('o', 'http://www.wikidata.org/entity/Q13235160'), ('oname', 'manufacturer')]
[('o', 'http://www.wikidata.org/entity/Q205375'), ('oname', 'inventor')]
[('o', 'http://www.wikidata.org/entity/Q2405480'), ('oname', 'voice actor')]
[('o', 'http://www.wikidata.org/entity/Q2526255'), ('oname', 'film director')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('oname', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3282637'), ('oname', 'film producer')]
[('o', 'http://www.wikidata.org/entity/Q33999'), ('oname', 'actor')]
[('o', 'http://www.wikidata.org/entity/Q3455803'), ('oname', 'director')]
[('o', 'http://www.wikidata.org/entity/Q36180'), ('oname', 'writer')]
[('o', 'http://www.wikidata.org/entity/Q644687'), ('oname', 'illustrator')]
[('o', 'http

## Task 4
List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

In [15]:
#we want to find appropriate predication for Walt Disney Animation Studios which is production company P272
queryString = """
SELECT DISTINCT  ?p ?pname 
WHERE {
   # bind something
    ?s ?p wd:Q1047410.
   # get the label
   ?p sc:name ?pname.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('pname', 'used by')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('pname', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of')]
[('p', 'http://www.wikidata.org/prop/direct/P199'), ('pname', 'business division')]
[('p', 'http://www.wikidata.org/prop/direct/P2650'), ('pname', 'interested in')]
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('pname', 'production company')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('pname', 'subsidiary')]
[('p', 'http://www.wikidata.org/prop/direct/P749'), 

In [13]:
# we want to know pulication date P577 of Lion King
queryString = """
SELECT DISTINCT  ?p ?pname ?o 
WHERE {
   # bind something
    wd:Q36479 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   #?o sc:name ?oname.
   filter regex(?pname,'publication','i')
   
}
LIMIT 15
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P577'), ('pname', 'publication date'), ('o', '1994-06-15T00:00:00Z')]
1


In [14]:
#we have found thae bdate and death date of walt desiny
queryString = """
SELECT DISTINCT ?p ?pname ?o
WHERE {
   # bind something
   wd:Q8704 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   filter regex(?pname, 'date')
}
LIMIT 15

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('pname', 'date of birth'), ('o', '1901-12-05T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('pname', 'date of death'), ('o', '1966-12-15T00:00:00Z')]
2


In [15]:

queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s wdt:P272 wd:Q1047410;
      wdt:P577  ?date.  
   # get the label
   ?s sc:name ?sname.
   FILTER(YEAR(?date) <= 1966 && YEAR(?date) >=1901)
}
LIMIT 15

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q215617'), ('sname', 'Sleeping Beauty')]
[('s', 'http://www.wikidata.org/entity/Q3236849'), ('sname', "Let's Stick Together")]
[('s', 'http://www.wikidata.org/entity/Q842306'), ('sname', 'Saludos Amigos')]
[('s', 'http://www.wikidata.org/entity/Q2090328'), ('sname', 'Trick or Treat')]
[('s', 'http://www.wikidata.org/entity/Q640679'), ('sname', 'Rugged Bear')]
[('s', 'http://www.wikidata.org/entity/Q2399982'), ('sname', 'Tea for Two Hundred')]
[('s', 'http://www.wikidata.org/entity/Q3028329'), ('sname', 'Two Weeks Vacation')]
[('s', 'http://www.wikidata.org/entity/Q3520106'), ('sname', 'The Brave Engineer')]
[('s', 'http://www.wikidata.org/entity/Q2700743'), ('sname', 'Clock Cleaners')]
[('s', 'http://www.wikidata.org/entity/Q3521221'), ('sname', 'The Haunted House')]
[('s', 'http://www.wikidata.org/entity/Q3036174'), ('sname', 'Grin and Bear It')]
[('s', 'http://www.wikidata.org/entity/Q3228508'), ('sname', 'Hold That Pose')]
[('s', 'http:

Final query for this task

## Task 5
Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

In [22]:
# write your queries
#we find the voice actors as a job by P106

queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   ?s wdt:P106 ?o;
      wdt:P31 wd:Q5.
   # get the label
   ?o sc:name ?oname.
   filter regex(?oname,'voice actor')
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2405480'), ('oname', 'voice actor')]
1


Final query for this task

In [11]:
# write your queries
# we want to know the name of people who has voive actor job Q2405480

queryString = """
SELECT DISTINCT ?name ?pname
WHERE {
   # bind something
    ?name wdt:P106 wd:Q2405480.
    # get the label
   ?name sc:name ?pname.
   
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'http://www.wikidata.org/entity/Q7670520'), ('pname', 'TJ Ramini')]
[('name', 'http://www.wikidata.org/entity/Q335080'), ('pname', 'Makoto Shinkai')]
[('name', 'http://www.wikidata.org/entity/Q13651397'), ('pname', 'Chan Hao Wai')]
[('name', 'http://www.wikidata.org/entity/Q15729747'), ('pname', 'Tim Moeseritz')]
[('name', 'http://www.wikidata.org/entity/Q15730711'), ('pname', 'Andressa Andreatto')]
[('name', 'http://www.wikidata.org/entity/Q16260148'), ('pname', 'Jane Liao')]
[('name', 'http://www.wikidata.org/entity/Q1689002'), ('pname', 'Jill Schulz')]
[('name', 'http://www.wikidata.org/entity/Q5072624'), ('pname', 'Chantal Strand')]
[('name', 'http://www.wikidata.org/entity/Q63284181'), ('pname', 'James Wootton')]
[('name', 'http://www.wikidata.org/entity/Q87192601'), ('pname', 'Rodrigo Andreatto')]
[('name', 'http://www.wikidata.org/entity/Q99906'), ('pname', 'Anne Helm')]
[('name', 'http://www.wikidata.org/entity/Q15875065'), ('pname', 'Eva Burmeister')]
[('name

In [29]:
# write your queries


queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
    ?name wdt:P106 wd:Q2405480;
           ?p ?film.
    ?film wdt:P272 wd:Q1047410.
   # get the label
   ?p sc:name ?pname.
   ?film sc:name ?fname.
     
}
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work')]
1


In [43]:
# write your queries
#this failed

queryString = """
SELECT DISTINCT ?name ?nname ?fname (count(?film)as ?number_of_film) 
WHERE {
   # bind something
    ?name wdt:P106 wd:Q2405480;
           wdt:P800 ?film.
    ?film wdt:P272 wd:Q1047410.
   # get the label
   ?name sc:name ?nname.
   ?film sc:name ?fname.
     
}
#group by ?name
Order by asc (?number_of_film)
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'http://www.wikidata.org/entity/Q1364909'), ('nname', 'Bill Farmer'), ('fname', 'A Goofy Movie'), ('number_of_film', '1')]
[('name', 'http://www.wikidata.org/entity/Q223110'), ('nname', 'John C. Reilly'), ('fname', 'Wreck-It Ralph'), ('number_of_film', '1')]
[('name', 'http://www.wikidata.org/entity/Q739105'), ('nname', 'Jason Marsden'), ('fname', 'A Goofy Movie'), ('number_of_film', '1')]
[('name', 'http://www.wikidata.org/entity/Q349350'), ('nname', 'Alan Tudyk'), ('fname', 'Wreck-It Ralph'), ('number_of_film', '1')]
[('name', 'http://www.wikidata.org/entity/Q203960'), ('nname', 'James Earl Jones'), ('fname', 'The Lion King'), ('number_of_film', '1')]
[('name', 'http://www.wikidata.org/entity/Q451833'), ('nname', 'Ernie Sabella'), ('fname', 'The Lion King'), ('number_of_film', '1')]
[('name', 'http://www.wikidata.org/entity/Q47541531'), ('nname', 'Any Gabrielly'), ('fname', 'Ralph Breaks the Internet'), ('number_of_film', '1')]
[('name', 'http://www.wikidata.org/ent

In [44]:
# write your queries


queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind something
   ?film wdt:P272 wd:Q1047410;
         ?p ?name.
   ?name wdt:P106 wd:Q2405480.
   # get the label
   ?name sc:name ?nname.
   ?p sc:name ?pname.
   ?film sc:name ?fname.
     
}
#group by ?name
Order by asc (?number_of_film)
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('pname', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pname', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('pname', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('pname', 'voice actor')]
[('p', 'http://www.wikidata.org/prop/direct/P3275'), ('pname', 'storyboard artist')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pname', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P6942'), ('pname', 'animator')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pname', 'screenwriter')]
[('p', 'http://www.wikidata.org/prop/direct/P2554'), ('pname', 'production designer')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('pname', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P1040')

In [71]:
# write your queries


queryString = """
SELECT DISTINCT ?actor ?actor_name (count(?actor)as ?number_of_film)
WHERE{
?film wdt:P725 ?actor.
filter(?actor =?name)
#filter(?number_of_film>4)
?actor sc:name ?actor_name.
    {SELECT DISTINCT ?film ?fname ?name ?nname
    WHERE {
       # bind something
       ?film wdt:P272 wd:Q1047410;
             wdt:P725 ?name.
        ?name wdt:P106 wd:Q2405480.
       # get the label
       ?name sc:name ?nname.
       ?film sc:name ?fname.
           }}
}
#group by ?name
Order by desc (?number_of_film>4)
limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q157400'), ('actor_name', 'Idina Menzel'), ('number_of_film', '5')]
[('actor', 'http://www.wikidata.org/entity/Q178882'), ('actor_name', 'Kristen Bell'), ('number_of_film', '6')]
[('actor', 'http://www.wikidata.org/entity/Q310292'), ('actor_name', 'Jim Cummings'), ('number_of_film', '10')]
[('actor', 'http://www.wikidata.org/entity/Q945591'), ('actor_name', 'Pinto Colvig'), ('number_of_film', '5')]
[('actor', 'http://www.wikidata.org/entity/Q264418'), ('actor_name', 'Russi Taylor'), ('number_of_film', '6')]
[('actor', 'http://www.wikidata.org/entity/Q349350'), ('actor_name', 'Alan Tudyk'), ('number_of_film', '7')]
[('actor', 'http://www.wikidata.org/entity/Q296577'), ('actor_name', 'Frank Welker'), ('number_of_film', '7')]
[('actor', 'http://www.wikidata.org/entity/Q847124'), ('actor_name', 'Maurice LaMarche'), ('number_of_film', '5')]
[('actor', 'http://www.wikidata.org/entity/Q847133'), ('actor_name', 'Fred Tatasciore'), ('number_of_

## Task 6
Identify the BGP for the film "The Little Mermaid"

In [12]:
# query example
# we find the film which walt disny is production company and with filter regex we find The Little Mermaid
queryString = """
SELECT DISTINCT ?s ?sname 
WHERE {
   # bind something
   ?s  wdt:P272 wd:Q1047410.
    # get the label
   ?s sc:name ?sname.
   filter regex(?sname,"The Little Mermaid")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q944287'), ('sname', 'The Little Mermaid')]
1


In [102]:
# query example
#bgp for The Little Mermaid
queryString = """
SELECT DISTINCT ?sname ?p ?pname ?o ?oname
WHERE {
   # bind something
   ?s  wdt:P272 wd:Q1047410;
        ?p   ?o.
   # get the label
   ?s sc:name ?sname.
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   filter regex(?sname,"The Little Mermaid")
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'The Little Mermaid'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q22812458'), ('oname', 'comedic television series')]
[('sname', 'The Little Mermaid'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pname', 'based on'), ('o', 'http://www.wikidata.org/entity/Q190135'), ('oname', 'The Little Mermaid')]
[('sname', 'The Little Mermaid'), ('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q190135'), ('oname', 'The Little Mermaid')]
[('sname', 'The Little Mermaid'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q1138602'), ('oname', 'Jodi Benson')]
[('sname', 'The Little Mermaid'), ('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator'), ('o', 'http://www.wikidata.org/entity/Q3565642'), ('oname', 'Walt Disney Television')]
[('sname', 'The Little Mermaid'), ('

Final query for this task

## Task 7
Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label).

In [105]:
# query example

queryString = """
SELECT DISTINCT  ?worker ?worker_name 
WHERE{
wd:Q944287 ?p ?worker.
?worker wdt:P31 wd:Q5.
filter(?worker=?worker_of_lionking)

?worker sc:name ?worker_name.
    {SELECT DISTINCT   ?worker_of_lionking 
    WHERE {
       # bind something
       wd:Q36479 ?p ?worker_of_lionking.
       ?worker_of_lionking wdt:P31 wd:Q5.
       # get the label
       }
    }
#?film sc:name ?film_name.
#filter regex(?film_name,"Mermaid")
}
group by (?film_name)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('worker', 'http://www.wikidata.org/entity/Q310292'), ('worker_name', 'Jim Cummings')]
1


Final query for this task

## Task 8
Identify the BGP for Academy Awards

In [39]:
# write your queries
#as we have found nominated for is P1411 

queryString = """
SELECT DISTINCT  ?o ?oname
WHERE {
   # bind something
   ?s  wdt:P1411 ?o.
   # get the label
   ?o sc:name ?oname.
   filter regex(?oname, 'Academy Awards')
}

"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q19020'), ('oname', 'Academy Awards')]
[('o', 'http://www.wikidata.org/entity/Q2004417'), ('oname', 'Indian Television Academy Awards')]
[('o', 'http://www.wikidata.org/entity/Q57246152'), ('oname', '93rd Academy Awards')]
[('o', 'http://www.wikidata.org/entity/Q18385710'), ('oname', 'Zulu African Film Academy Awards')]
4


Final query for this task

In [107]:
# write your final query
#I find bgp for award

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q19020 ?p ?o.
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q102427'), ('oname', 'Academy Award for Best Picture')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q103360'), ('oname', 'Academy Award for Best Director')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q103618'), ('oname', 'Academy Award for Best Actress')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q106301'), ('oname', 'Academy Award for Best Supporting Actress')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q107258'), ('oname', 'Academy Award for Best Writing, Adapted Screenplay')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1

## Task 9
For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

In [110]:
# query example
#i find nominated for
queryString = """
SELECT DISTINCT ?p ?pname ?oname
WHERE {
   # bind something
   wd:Q36479 ?p ?o .
   # get the label
   ?p sc:name ?pname.
   ?o sc:name ?oname.
   filter regex(?pname, 'nominated')

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for'), ('oname', 'Academy Award for Best Original Score')]
1


In [16]:
# walt Q1047410
#once I considered wdt:P527 but it was not approprate predicate so I used P31 
queryString = """
SELECT DISTINCT ?film ?film_name  (count(?award)as ?number_of_award)
WHERE {
?film wdt:P1411 ?award.
?award wdt:P31 wd:Q19020.
#wd:Q19020 wdt:P527 ?award.
filter(?film = ?waltdisney_films)
?film sc:name ?film_name.
?award sc:name ?aname.
    {SELECT DISTINCT ?waltdisney_films ?wname
    WHERE {
    # bind something
    ?waltdisney_films wdt:P272 wd:Q1047410.
     # get the label
    ?waltdisney_films sc:name ?wname.
    }}
    }
#group by()
Order by desc(?number_of_award)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q379873'), ('film_name', 'Who Framed Roger Rabbit'), ('number_of_award', '6')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('film_name', 'Beauty and the Beast'), ('number_of_award', '3')]
[('film', 'http://www.wikidata.org/entity/Q842306'), ('film_name', 'Saludos Amigos'), ('number_of_award', '2')]
[('film', 'http://www.wikidata.org/entity/Q13091172'), ('film_name', 'Big Hero 6'), ('number_of_award', '1')]
[('film', 'http://www.wikidata.org/entity/Q36479'), ('film_name', 'The Lion King'), ('number_of_award', '1')]
[('film', 'http://www.wikidata.org/entity/Q213787'), ('film_name', 'The Hunchback of Notre Dame'), ('number_of_award', '1')]
[('film', 'http://www.wikidata.org/entity/Q12124484'), ('film_name', 'Get a Horse!'), ('number_of_award', '1')]
[('film', 'http://www.wikidata.org/entity/Q1440760'), ('film_name', 'Runaway Brain'), ('number_of_award', '1')]
[('film', 'http://www.wikidata.org/entity/Q3259450'), ('film_name', 'Lor

Final query for this task

In [ ]:
# write your final query